In [ ]:
from hana_ml import dataframe
from hana_ml.algorithms.pal.utility import DataSets
from gen_ai_hub.proxy.langchain import init_llm
#from gen_ai_hub.proxy.langchain.init_models import init_llm
from hana_ai.vectorstore.hana_vector_engine import HANAMLinVectorEngine
from hana_ai.vectorstore.code_templates import get_code_templates
llm = init_llm('gpt-4', temperature=0.0, max_tokens=512)

In [ ]:

connection_context = dataframe.ConnectionContext(userkey="RaysKey")
hana_df, _, _, _ = DataSets.load_iris_data(connection_context)
hana_df = hana_df.to_head("ID")

Table IRIS_DATA_FULL_TBL exists.


In [10]:
from hana_ai.tools.code_template_tools import GetCodeTemplateFromVectorDB

hana_vec = HANAMLinVectorEngine(connection_context, "hana_vec_hana_ml_knowledge")
hana_vec.create_knowledge()



code_tool = GetCodeTemplateFromVectorDB()
code_tool.set_vectordb(hana_vec)

100%|██████████| 1/1 [00:21<00:00, 21.64s/it]


In [4]:
connection_context.table("hana_vec_hana_ml_knowledge").head(10).collect()

,id,description,example,embeddings
0,abc_analysis,The abc_analysis function in the hana_ml.algor...,\nHere is a Python code template based on the ...,"[-0.02208906039595604, 0.04436293989419937, -0..."
1,accuracy_measure,The accuracy_measure function in the hana_ml.a...,\nHere is the executable code template for the...,"[0.004033094272017479, 0.04234658554196358, -0..."
2,accuracy_score,The accuracy_score function in the hana_ml.alg...,\nHere is a Python code template based on the ...,"[0.008136419579386711, 0.029762445017695427, -..."
3,AdditiveModelForecastExplainer,The AdditiveModelForecastExplainer class in th...,\nHere is a Python code template for the `Addi...,"[-0.01599763333797455, -0.0254523828625679, 0...."
4,AffinityPropagation,The AffinityPropagation class in the hana_ml.a...,\nHere is a Python code template based on the ...,"[0.010633992031216621, -0.016167793422937393, ..."
5,AgglomerateHierarchicalClustering,The AgglomerateHierarchicalClustering class in...,\nHere is a Python code template based on the ...,"[-0.026697132736444473, -0.014196800999343395,..."
6,AMDPDeployer,The AMDPDeployer class in the hana_ml.artifact...,\nHere is a Python code template based on the ...,"[0.003183638211339712, -0.03143170103430748, -..."
7,AMDPGenerator,The AMDPGenerator class in the hana_ml.artifac...,\nHere is a Python code template based on the ...,"[-0.021213607862591743, -0.013314641080796719,..."
8,Apriori,The Apriori class in the hana_ml.algorithms.pa...,\nHere is a Python code template based on the ...,"[0.023184102028608322, 0.0018620929913595319, ..."
9,AprioriLite,The AprioriLite class in the hana_ml.algorithm...,\nHere is a Python code template based on the ...,"[0.01068233884871006, -0.0018579716561362147, ..."


In [6]:
from hana_ai.smart_dataframe import SmartDataFrame

sdf = SmartDataFrame(hana_df)

In [11]:
sdf.configure(tools=[code_tool], llm=llm)

In [12]:
sdf.ask("Create dataset report", verbose=True)



> Entering new AgentExecutor chain...
Thought: To create a dataset report, I need to use the `create_report` function from the `Dataframe` class in the `hana_ml` library. This function generates a report about the dataset, including statistics and visualizations. I need to call this function on the `df` dataframe.
Action: python_repl_ast
Action Input: df.create_report()
Observation: AttributeError: __getattr__ argument not understood: 'create_report'
Thought:It seems like the `create_report` function is not available in the `Dataframe` class of the `hana_ml` library. I need to find an alternative way to create a dataset report. I can use the `describe` function to get the statistical summary of the dataframe. This function returns count, mean, std, min, 25%, 50%, 75%, max for each numerical column in the dataframe.
Action: python_repl_ast
Action Input: df.describe().collect()
Observation:           column  count  unique  nulls       mean        std  min    max  \
0             ID    

{'input': 'Create dataset report',
 'output': 'The dataset report is as follows:\n\nID: count=150, unique=150, nulls=0, mean=74.5, std=43.44, min=0, max=149, median=75, 25%=37.25, 50%=74.5, 75%=111.75\nSEPALLENGTHCM: count=150, unique=35, nulls=0, mean=5.84, std=0.82, min=4.3, max=7.9, median=5.8, 25%=5.1, 50%=5.8, 75%=6.4\nSEPALWIDTHCM: count=150, unique=23, nulls=0, mean=3.05, std=0.43, min=2, max=4.4, median=3, 25%=2.8, 50%=3, 75%=3.3\nPETALLENGTHCM: count=150, unique=43, nulls=0, mean=3.75, std=1.76, min=1, max=6.9, median=4.35, 25%=1.6, 50%=4.35, 75%=5.1\nPETALWIDTHCM: count=150, unique=22, nulls=0, mean=1.19, std=0.76, min=0.1, max=2.5, median=1.3, 25%=0.3, 50%=1.3, 75%=1.8\nSPECIES: count=150, unique=3, nulls=0, mean=N/A, std=N/A, min=N/A, max=N/A, median=N/A, 25%=N/A, 50%=N/A, 75%=N/A'}

In [13]:
connection_context.drop_table("my_model")

ERROR:hana_ml.dataframe:Fail to drop table. (259, 'invalid table name: my_model: line 1 col 25 (at pos 24)')


In [14]:
sdf.ask("Create Automatic Classification model on this dataframe with max_eval_time_mins=10. Provide key is ID, background_size is 100, label is SPECIES and model table is 'my_model' in the fit function and execute it. ", verbose=True)



> Entering new AgentExecutor chain...
Thought: To create an Automatic Classification model, I need to import the necessary module from hana_ml and instantiate the model. Then, I need to fit the model to the dataframe using the specified parameters.
Action: CodeTemplatesFromVectorDB
Action Input: Create Automatic Classification model
Observation: 
Strictly use the Examples to create AutoML instance.
Do fitting:
>>> from hana_ml.algorithms.pal.auto_ml import AutomaticClassification
>>> mymodel = AutomaticClassification(generations=2, population_size=5, offspring_size=5, max_eval_time_mins=10, successive_halving=True)
>>> mymodel.enable_workload_class(<WORKLOAD_CLASS>)
>>> mymodel.fit(data=<data_frame>, key=<ID>, model_table_name=<MODEL_TABLE>, background_size=<background_size>)
Thought:

ERROR:hana_ml.ml_exceptions:Please define the workload class and call the enable_workload_class() method first!
ERROR:hana_ml.ml_exceptions:Please define the workload class and call the enable_workload_class() method first!


I now know how to create an Automatic Classification model and fit it to the dataframe. I need to replace the placeholders in the code with the actual values.
Action: python_repl_ast
Action Input: 
from hana_ml.algorithms.pal.auto_ml import AutomaticClassification
mymodel = AutomaticClassification(generations=2, population_size=5, offspring_size=5, max_eval_time_mins=10, successive_halving=True)
mymodel.fit(data=df, key='ID', model_table_name='my_model', background_size=100, label='SPECIES')
Observation: FitIncompleteError: Please define the workload class and call the enable_workload_class() method first!
Thought:I forgot to enable the workload class before fitting the model. I need to call the enable_workload_class() method and provide a workload class.
Action: python_repl_ast
Action Input: 
mymodel.enable_workload_class('XS')
mymodel.fit(data=df, key='ID', model_table_name='my_model', background_size=100, label='SPECIES')
Observation: <hana_ml.algorithms.pal.auto_ml.AutomaticClassif

{'input': "Create Automatic Classification model on this dataframe with max_eval_time_mins=10. Provide key is ID, background_size is 100, label is SPECIES and model table is 'my_model' in the fit function and execute it. ",
 'output': 'The Automatic Classification model has been created and fitted to the dataframe with the specified parameters.'}

In [15]:
connection_context.table("my_model").collect()

,ROW_INDEX,MODEL_CONTENT
0,0,"{""pipeline"":{""MLP_Classifier"":{""args"":{""ACTIVA..."
1,1,"""ConnectionsFrom\"":[{\""from\"":0,\""weight\"":0.2..."
2,2,"12990443882,\""ConnectionsFrom\"":[{\""from\"":0,\..."
3,3,""":59,\""bias\"":-0.6866103229717347,\""Connection..."
4,4,"7366527}]},{\""id\"":80,\""bias\"":-0.632423581087..."
5,5,"\"":-0.37678682407273847}]},{\""id\"":101,\""bias\..."
6,6,".12251929088685923},{\""from\"":96,\""weight\"":0...."
7,7,"106,\""bias\"":-0.09439707779298194,\""Connection..."
8,8,""":\""SPECIES\"",\""value\"":\""Iris-virginica\""}}}]..."


In [16]:
sdf.ask("Show the samples of the dataset", verbose=True)



> Entering new AgentExecutor chain...
Thought: To show the samples of the dataset, I need to use the `head()` function on the dataframe `df`.
Action: python_repl_ast
Action Input: df.head().collect()
Observation:    ID  SEPALLENGTHCM  SEPALWIDTHCM  PETALLENGTHCM  PETALWIDTHCM      SPECIES
0   0            5.1           3.5            1.4           0.2  Iris-setosa
Thought:I now know the final answer
Final Answer: The samples of the dataset are as follows:
   ID  SEPALLENGTHCM  SEPALWIDTHCM  PETALLENGTHCM  PETALWIDTHCM      SPECIES
0   0            5.1           3.5            1.4           0.2  Iris-setosa

> Finished chain.


{'input': 'Show the samples of the dataset',
 'output': 'The samples of the dataset are as follows:\n   ID  SEPALLENGTHCM  SEPALWIDTHCM  PETALLENGTHCM  PETALWIDTHCM      SPECIES\n0   0            5.1           3.5            1.4           0.2  Iris-setosa'}

In [17]:
new_df = sdf.transform("Get first two rows", verbose=True)



> Entering new AgentExecutor chain...
Thought: To get the first two rows, I need to use the head() function with parameter 2. Then, I need to save the result to trans_df and return the SQL string from calling trans_df.select_statement.

Action: python_repl_ast
Action Input: 
```python
trans_df = df.head(2)
sql_string = trans_df.select_statement
sql_string
```
Observation: SELECT TOP 2 * FROM (SELECT "ID", "SEPALLENGTHCM", "SEPALWIDTHCM", "PETALLENGTHCM", "PETALWIDTHCM", "SPECIES" FROM (SELECT * FROM "PALDEVUSER"."IRIS_DATA_FULL_TBL") AS "DT_2") dt
Thought:Final Answer: SELECT TOP 2 * FROM (SELECT "ID", "SEPALLENGTHCM", "SEPALWIDTHCM", "PETALLENGTHCM", "PETALWIDTHCM", "SPECIES" FROM (SELECT * FROM "PALDEVUSER"."IRIS_DATA_FULL_TBL") AS "DT_2") dt

> Finished chain.


In [18]:
new_df.shape

[2, 6]

In [19]:
import pandas as pd
external_knowledge = get_code_templates(customized_dir="external_knowledge")

external_knowledge

{'id': ['HANAGeneratorForCAP', 'Teched_codesnippet1', 'Teched_codesnippet2'],
 'description': ['The HANAGeneratorForCAP class in the hana_ml.artifacts.generators.hana module is a HANA artifacts generator for existing CAP projects, which takes a project name, output directory, and optional namespace as parameters, and includes methods to generate CAP artifacts and create an input table for the input dataframe.\n',
  'useful for when you need to create a time series forecast model for the table SALES_REFUNDS include model statistics and save the model.\n',
  'useful for when you need to calculate predictions and show the results in a forecast line plot.\n'],
 'example': ['\nHere is a Python code template based on the provided help doc:\n\n```python\nfrom hana_ml.artifacts.generators.hana import HANAGeneratorForCAP\nfrom hana_ml.algorithms.pal.tsa.additive_model_forecast import AdditiveModelForecast\nfrom hana_ml.visualizers.visualizer_base import forecast_line_plot\nfrom hana_ml.datafram

In [20]:
second_hana_vec = HANAMLinVectorEngine(connection_context, "second_hana_vec_hana_ml_knowledge")
second_hana_vec.upsert_knowledge(external_knowledge)

100%|██████████| 1/1 [00:00<00:00,  1.83it/s]


In [21]:
from hana_ai.vectorstore.union_vector_stores import UnionVectorStores

uvs = UnionVectorStores([second_hana_vec, hana_vec])


In [22]:
code_tool.set_vectordb(uvs)

In [23]:
ts_df = DataSets.load_shampoo_data(connection_context)

Table SHAMPOO_SALES_DATA_TBL exists.


In [24]:
s_ts_df = SmartDataFrame(ts_df)
s_ts_df.configure(tools=[code_tool], llm=llm)

In [25]:
s_ts_df.head(1).collect()

,ID,SALES
0,1,266.0


In [26]:
s_ts_df.ask("Create Automatic TimeSeires model on this dataframe. Provide key is ID, endog is SALES and model table is 'my_model_ts' in the fit function and execute it. ", verbose=True)



> Entering new AgentExecutor chain...
Thought: To create an Automatic TimeSeries model, I need to import the necessary module from hana_ml and instantiate the model. Then, I need to fit the model to the dataframe using the specified parameters.
Action: CodeTemplatesFromVectorDB
Action Input: Create Automatic TimeSeries model
Observation: 
Here is a Python code template based on the provided help doc:

```python
from hana_ml.algorithms.pal.tsa.additive_model_forecast import AdditiveModelForecast
from hana_ml.visualizers.visualizer_base import forecast_line_plot
from hana_ml.dataframe import DataFrame

# sort the dataframe 
<dataframe> =  <datarame>.sort(['BOOKING_DATE'])

# train the model
amf = AdditiveModelForecast(growth='linear')
amf.fit(data=<dataframe>.head(150))
pred_data = amf.predict(data=<dataframe>.tail(30))

#save the model table
amf.model_.save('SALES_REFUNDS_MODEL', force=True)

# calculate the model statistics
test_gt=<dataframe>.sort('BOOKING_DATE', desc=True).rename_c

{'input': "Create Automatic TimeSeires model on this dataframe. Provide key is ID, endog is SALES and model table is 'my_model_ts' in the fit function and execute it. ",
 'output': "The AutoARIMA model has been successfully created, fitted to the dataframe, and saved with the name 'my_model_ts'."}

In [27]:
connection_context.table("my_model_ts").collect()

,KEY,VALUE
0,p,1
1,AR,-0.567381
2,d,1
3,q,1
4,MA,-0.51326
5,s,2
6,P,0
7,SAR,
8,D,0
9,Q,0
